In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df_links = pd.read_csv("../input/100honch8/links.csv")
df_links

In [3]:
import matplotlib.pyplot as plt
import networkx as nx

#creating Graph object
G = nx.Graph()

#setting nodes
NUM = len(df_links.index)
for i in range(1,NUM+1):
    node_no = df_links.columns[i].strip("Node")
    G.add_node(str(node_no))

#setting edges
for i in range(NUM):
    for j in range(NUM):
        node_name = "Node" + str(j)
        if df_links[node_name].iloc[i]==1:
            G.add_edge(str(i),str(j))
            
#plot my graph
#draw_networkx : Automatically determine the position of nodes so that those with the most links are clustered in the center
nx.draw_networkx(G,node_color='k',edge_color='k',font_color='w')
plt.show()

In [4]:
#simulate spread around my network

#determine probability if the pulse will spread
def determine_link(percent):
    rand_val = np.random.rand()
    if rand_val <= percent:
        return 1
    else:
        return 0
    
    
#num: sum of nodes, list_active: lists of every-node activation, percent_percolation: probability of pulses occurring
#O(N^2)
def simulation_percolation(num, list_active, percent_percolation):
    for i in range(num):
        if list_active[i]==1:
            for j in range(num):
                node_name = "Node" + str(j)
                if df_links[node_name].iloc[i]==1:
                    if determine_link(percent_percolation)==1:
                        list_active[j] = 1
    return list_active


#initialization, setting parameters
percent_percolation = 0.1
T_NUM = 100 #time length of simulation
NUM = len(df_links.index)
list_active = np.zeros(NUM)
list_active[0] = 1
list_timeSeries = []

for t in range(T_NUM):
    list_active = simulation_percolation(NUM, list_active, percent_percolation)
    list_timeSeries.append(list_active.copy())


#network visualization

def active_node_coloring(list_active):
    list_color=[]
    for i in range(len(list_timeSeries[t])):
        if list_timeSeries[t][i]==1:
            list_color.append("r")
        else:
            list_color.append("k")
    return list_color

In [5]:
t=0
nx.draw_networkx(G, font_color='w', node_color=active_node_coloring(list_timeSeries[t]))
plt.show()

In [6]:
t=10
nx.draw_networkx(G, font_color='w', node_color=active_node_coloring(list_timeSeries[t]))
plt.show()

In [7]:
t=25
nx.draw_networkx(G, font_color='w', node_color=active_node_coloring(list_timeSeries[t]))
plt.show()

In [8]:
t=50
nx.draw_networkx(G, font_color='w', node_color=active_node_coloring(list_timeSeries[t]))
plt.show()

In [9]:
#Time series graph of the number of active nodes.
list_timeSeries_num = []

flag = False
for i in range(len(list_timeSeries)):
    list_timeSeries_num.append(sum(list_timeSeries[i]))
    if sum(list_timeSeries[i])==NUM and not flag:
        print(i)
        flag=True
    
plt.plot(list_timeSeries_num)
plt.show()

In [10]:
#more general simulation

def simulate_population(num, list_active, percent_percolation, percent_disapparence, df_links):
    #spread
    for i in range(num):
        if list_active[i]==1:
            for j in range(num):
                if df_links.iloc[i][j]==1:
                    if determine_link(percent_percolation)==1:
                        list_active[j]=1
    
    #disapparence
    for i in range(num):
        if determine_link(percent_disapparence)==1:
            list_active[i]==0
    
    return list_active


#initialize

precent_percolation = 0.08
percent_disapparence = 0.2
T_NUM = 100

NUM = len(df_links.index)
list_active = np.zeros(NUM)
list_active[0] = 1

list_timeSeries = []


#simulate & plot
for t in range(T_NUM):
    list_active = simulate_population(NUM, list_active, percent_percolation, percent_disapparence, df_links)
    list_timeSeries.append(list_active.copy())
    
list_timeSeries_num = []
for i in range(len(list_timeSeries)):
    list_timeSeries_num.append(sum(list_timeSeries[i]))
    
plt.plot(list_timeSeries_num)
plt.show()

In [12]:
#simulate for 100 months
T_NUM = 100
NUM_PhaseDiagram = 20
phaseDiagram = np.zeros((NUM_PhaseDiagram, NUM_PhaseDiagram))

for i_p in range(NUM_PhaseDiagram):
    for i_d in range(NUM_PhaseDiagram):
        percent_percolation = 0.05*i_p
        percent_disapparence = 0.05*i_d
        list_active = np.zeros(NUM)
        list_active[0] = 1
        for t in range(T_NUM):
            list_active = simulate_population(NUM, list_active, percent_percolation, percent_disapparence, df_links)
        phaseDiagram[i_p][i_d] = sum(list_active)
        
        
plt.matshow(phaseDiagram)
plt.colorbar(shrink=0.8)
plt.xlabel('percent_disapparence')
plt.ylabel('percent_percolation')
plt.xticks(np.arange(0,20,5), np.arange(0,1,0.25))
plt.yticks(np.arange(0,20,5), np.arange(0,1,0.25))
plt.tick_params(bottom=False, left=False, right=False, top=False)
plt.show()